# re --- 正規表現操作
### Agenda

- 正規表現の仕組みとPythonでの利用の仕方をまとめる

### 参考

- https://docs.python.org/ja/3/library/re.html


In [1]:
## create the clean environment
import gc
import matplotlib.pyplot as plt

def clear_all():
    # Clears all the variables from the workspace
    gl = globals().copy()
    for var in gl:
        if var in clean_env_var: continue
        del globals()[var]
    # Garbage collection:
    gc.collect()

def close_plots():
  my_plots = plt.get_fignums()
  for j in my_plots:
    plt.close(plt.figure(j))

clean_env_var = dir()
clean_env_var.append('clean_env_var')

In [2]:
clear_all()

### Hardware

In [3]:
%%bash
system_profiler SPHardwareDataType | grep -E \
"Model Identifier"\|"Processor Name"\|"Processor Speed"\
\|"Number of Processors"\|"Memory:"

      Model Identifier: MacBookPro13,1
      Processor Name: Dual-Core Intel Core i5
      Processor Speed: 2 GHz
      Number of Processors: 1
      Memory: 16 GB


### Python

In [4]:
!python -V

Python 3.7.4


### Install Libraries

In [5]:
pass

### Import

In [6]:
import re

## 1. 正規表現入門

- 正規表現はregular expressionの訳語
- 元々は文字列のパターンを記述するための表現式として誕生
- 検索、置換、抽出、削除といった文字列処理をする際に重宝される


### 正規表現の基本

```
0|1|2|3|4|5|7|8|9
```

という正規表現は「0から9までの数字一文字」というパターンを表現している。

- `|`: orを表す演算子
- `[0-9]`, `\d`も「0から9までの数字一文字」

```
03-[0-9][0-9][0-9][0-9]-[0-9][0-9][0-9][0-9]
```

という正規表現は東京都の電話番号のパターンを示す

- `03-[0-9]{4}-[0-9]{4}`も同じ
- `03(-[0-9]{4}){2}`も同じ
- `03(-\d{4}){2}`も同じ
- `{4}`は「繰り返し」に関する演算子（量指定子,quantifier）


### 基本原則

- 連結
- 選択
- 繰り返し


### メタ文字とリテラル

- メタ文字: 正規表現のおいて特別な意味を持った文字
- リテラル: 正規表現のおいて特別な意味を持たない文字



### 文字列処理の一例

- 置換

```
sed -i -E 's/^(maxClients)*[0-9] + /\1 2048/' hoge.txt
```

- 検索

```
grep -E '正規表現' source.txt
```

### 正規表現の例

- `(...)?`: カッコ内か空文字列
- `\d*`: 任意の長さの数字列

#### Pythonの識別子に使える文字列の命名規則

- 先頭がアルファベットかアンダースコアで始まっている
- 先頭がアルファベットかアンダースコア, 数字の組み合わせで作れる文字列


これを正規表現で記述すると
```
[a-zA-Z_][a-zA-Z0-9_]*
```


### 演算子の結合順位

```
繰り返し > 連結 > 選択
```

- 丸括弧によるグループ化を用いることでどの演算を優先的に評価するかを明示的に指定することができる

### 正規表現のシンタックスシュガー

- `+`: 一回上の任意回の繰り返し
- `?`: あるか無いか、0回か1回か
- `{n, m}`: n回以上m回以下の繰り返し
- `.`: 任意の一文字
- `[^]`: 文字クラスにおける`[]`内部では先頭の`^`は否定の記号として特別扱いされる

### 文字クラス

- `[0-9]`: 0-9までの数字
- `[a-z]`: 小文字のみのアルファベット
- `[A-Z]`: 大文字のみのアルファベット
- `[a-zA-Z]`: アルファベット（大文字小文字含む）

`[0-9A-F]`とかくと16進数が抽出できる
 


### エスケープシーケンス

- `\t`: tab文字
- `\n`: 改行文字
- `\d`: `[0-9]`
- `\w`:`[0-9a-zA-Z_]`
- `\D`: `[^0-9]`

処理系によって使用可能なエスケープシーケンスは異なることに注意

### アンカー

アンカーとは文字列ではなく位置にマッチするメタ文字

- `^`：先頭
- `$`：行末
- `\A`: テキスト先頭


### prefix, substring, suffix

長さnの文字列が与えられた時

- prefix: n個
- suffix: n個
- substring: 空文字含めてn(n+1)/2 + 1個


#### Pythonと前方一致と部分一致

- `re.match`: 前方一致
- `re.search`: 部分一致

In [11]:
test = 'presubsuf'
True if re.match('sub', test) else False

False

In [12]:
True if re.search('sub', test) else False

True

### キャプチャ

正規表現と文字列からサブマッチを取得することをキャプチャと呼ぶ

In [42]:
r = re.compile("(\d{2})/(\d{2})/(\d{4})")
r.match("11/14/1980").groups()

('11', '14', '1980')

### 例題１

`ab+c`と`ab*c`の違い

#### 解

`ab+c`だと`ac`にはマッチしないが、`ab*c`だとする

### 例題２

`https?://gihyo.jp/`という正規表現でマッチする文字列は何か？

#### 解

- `https://gihyo.jp/`
- `http://gihyo.jp/`


### 例題3

```
03-4567i4-567u875
03-4567-5678
0345675678
りんご
```
から東京の電話番号表記に沿った文字列を抽出しろ


In [36]:
%%bash
input="03-4567i4-567u875
03-4567-5678
0345675678
りんご"

echo "$input"

03-4567i4-567u875
03-4567-5678
0345675678
りんご


In [39]:
%%bash
input="03-4567i4-567u875
03-4567-5678
0345675678
りんご"

echo "$input" | egrep '03(-[0-9]{4}){2}'

03-4567-5678
